# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-03-27*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [12]:
uvwind = xr.open_dataset('F:/Data/case_anqing/COMBINE_ACONC_CN3AH_135X138_2023_uvwind.nc')
met = xr.open_dataset('F:/Data/case_anqing/COMBINE_ACONC_CN3AH_135X138_2023_met.nc')
grid = xr.open_dataset('F:/Data/case_anqing/GRIDCRO2D_2023021.nc')

In [6]:
# 将时间点转换为实际时间
times=pd.date_range('2023-02-01-00','2023-02-28-23',freq='h')
times

DatetimeIndex(['2023-02-01 00:00:00', '2023-02-01 01:00:00',
               '2023-02-01 02:00:00', '2023-02-01 03:00:00',
               '2023-02-01 04:00:00', '2023-02-01 05:00:00',
               '2023-02-01 06:00:00', '2023-02-01 07:00:00',
               '2023-02-01 08:00:00', '2023-02-01 09:00:00',
               ...
               '2023-02-28 14:00:00', '2023-02-28 15:00:00',
               '2023-02-28 16:00:00', '2023-02-28 17:00:00',
               '2023-02-28 18:00:00', '2023-02-28 19:00:00',
               '2023-02-28 20:00:00', '2023-02-28 21:00:00',
               '2023-02-28 22:00:00', '2023-02-28 23:00:00'],
              dtype='datetime64[ns]', length=672, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [ ]:
# calculate saturation vapor pressure (es)
es = 6.112 * np.exp((17.67 * met.SFC_TMP) / (met.SFC_TMP + 243.5))

# calculate vapor pressure (e)
e = met.PRES * met.QV / (0.622 + 0.378 * met.QV)

# calculate relative humidity (RH)
RH = e / es * 100

In [16]:
days=11 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from mcip
        QV=(['time','level','y','x'],met.QV[days*24-8:-8-1,:,:,:].data,{'long name':'Water Vapor Mixing Ratio','units':'kg kg-1'}),
        RH=(['time','level','y','x'],RH.data,{'long name':'Relative Humidity on Surface','units':'%'}),
        SFC_TMP=(['time','level','y','x'],met.SFC_TMP[days*24-8:-8-1,:,:,:].data,{'long name':'Surface Temperature','units':'deg C'}),
        AIR_TMP=(['time','level','y','x'],met.AIR_TMP[days*24-8:-8-1,:,:,:].data,{'long name':'Air Temperature','units':'deg C'}),
        PBLH=(['time','level','y','x'],met.PBLH[days*24-8:-8-1,:,:,:].data,{'long name':'Planet Boundary Layer Height','units':'m'}),
        SOL_RAD=(['time','level','y','x'],met.SOL_RAD[days*24-8:-8-1,:,:,:].data,{'long name':'Solar Radiation','units':'W m-2'}),
        PRES=(['time','level','y','x'],met.PRES[days*24-8:-8-1,:,:,:].data,{'long name':'Air Pressure','units':'hPa'}),
        precip=(['time','level','y','x'],met.precip[days*24-8:-8-1,:,:,:].data,{'long name':'Precipitation','units':'cm'}),
        WSPD10=(['time','level','y','x'],met.WSPD10[days*24-8:-8-1,:,:,:].data,{'long name':'Wind Speed 10m','units':'m s-1'}),
        WDIR10=(['time','level','y','x'],met.WDIR10[days*24-8:-8-1,:,:,:].data,{'long name':'Wind Direction','units':'deg'}),
        CloudFRAC=(['time','level','y','x'],met.CloudFRAC[days*24-8:-8-1,:,:,:].data,{'long name':'Cloud Fraction','units':'1'}),
        uwind=(['time','level','y','x'],uvwind.UWind[days*24-8:-8-1,:,:-1,:-1].data,{'long name':'U-direction Horizontal Wind Speed','units':'m s-1'}),
        vwind=(['time','level','y','x'],uvwind.VWind[days*24-8:-8-1,:,:-1,:-1].data,{'long name':'V-direction Horizontal Wind Speed','units':'m s-1'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202302',
        grid='CN3AH_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 672, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-02-01 ... 2023-02-28T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    QV         (time, level, y, x) float32 0.003203 0.003225 ... 1.882e-06
    SFC_TMP    (time, level, y, x) float32 10.86 11.94 10.03 ... -273.1 -273.1
    AIR_TMP    (time, level, y, x) float32 13.65 13.07 12.06 ... -67.46 -67.52
    PBLH       (time, level, y, x) float32 55.69 94.2 95.75 ... -67.46 -67.52
    SOL_RAD    (time, level, y, x) float32 0.0 0.0 0.0 ... -67.43 -67.46 -67.52
    PRES       (time, level, y, x) float32 970.8 940.7 917.7 ... 61.69 61.69
    precip     (time, level, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WSPD10     (time, level, y, x) float32 2.718 4.926 4.466 ... 0.0 0.0 0.0
    WDIR10     (time, level, y, x) float32 136.0 148.7 188.8 ... 0.0 0.0 0.0
    CloudFRAC  (time, level, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    uwind      (time, level, y, x) float32 -1.795 -3.269 -1.608 ... 18.46 18.47
    vwind      (time, level, y, x) float32 3.723 4.06 3.498 ... 4.0 4.0 4.01
Attributes:
    case:     Anqing_202302
    grid:     CN3AH_135X138

In [18]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/compressed_met.nc',encoding=encoding)